# Overview
In this notebook I am going to describe how in general R-CNN and its children (Fast R-CNN and Faster R-CNN) work.
My describtion will be based on part of Udacity Computer Vision Nanodegree and those blogs: [one](https://towardsdatascience.com/deep-learning-for-object-detection-a-comprehensive-review-73930816d8d9), [two](https://www.learnopencv.com/selective-search-for-object-detection-cpp-python/), [three](https://qph2.c7.quoracdn.net/main-qimg-3fa20d817a121d31c641978114ff6f5d.webp).

### R-CNN

Region-based Convolutional Neural Network consists of 2 main steps:
#### 1. Region proposal algorithm is used to generate many areas where objects may be found
<img src="images/RegionProposal.png"><br>
Credits: Udacity Computer Vision Nanodegree<br><br>
**Q1**: But what is and how does **region proposal algorithm** work?<br>
**A1**: These methods takes image as an input and then output bounding boxes(coordinates) of all chunks of image which has high chance to be an object. Having extracted those areas we can then perform iamge recognition to determine what excacly that object is. This family of algorithms identify objects through **segmentation**. In segmentation, adjacent regions of image similar to each other (based on some criteria like color, gradient, texture etc.) are grouped. Also, those algorithms have a very high **recall** which means that we have many false positives detected (background), but almost for sure we have detected all true positives. In this case having many false positves is not a problem because they are going to be rejected during obejct recognition phase.<br>
**Example of such algorithm is Selective Search described comprehensively in blog [two](https://www.learnopencv.com/selective-search-for-object-detection-cpp-python/)**.<br><br>
#### 2. Each of proposed areas(region of interest = RoI) goes through classification CNN which outputs a class label and bounding box
<img src="images/RoItoCNN.png"><br>
Credits: Udacity Computer Vision Nanodegree<br><br>
Every RoI goes through CNN and the class label is predicted (for example DOG). In this case, class **backgroud** is included in order to capture any noisy regions. During training such model loss function is a weighted average of loss for classification (predicting classes) and regression(predicting coordinates for bounding boxes) problems.<br><br>

**Q2**: What was the biggest disadvantage of that method?<br>
**A2**: When processing one image, CNN has to be run on every RoI which drasticly increases time of both training and detecting.


### Fast R-CNN
R-CNN is accurate but quite slow and inefficient approach. In order to address that **Fast R-CNN** came. The main difference is that Fast R-CNN runs entire image through the CNN only once!<br>
**Q3**: But how is this possible?<br>
**A3**: RoI pooling layers were introduced. Their task is to take as an input feature map of given image, and then for that feature map for every RoI then perform max pooling in such a way that the output matches some pre-defined value (eg. 2x2)
<img src="https://cdn-sv1.deepsense.ai/wp-content/uploads/2017/02/roi_pooling-1.gif"><br>
Credits: [deepsense](https://deepsense.ai/region-of-interest-pooling-explained/)<br><br>

This approach is much faster then R-CNN but still uses region proposal algortihm for detecting potencial objects.

### Faster R-CNN
Here we finally replace slow region proposal algorithm(eg. selective search) with **region proposal network**.<br>

<img src="https://cdn-images-1.medium.com/max/1000/1*7heX-no7cdqllky-GwGBfQ.png"><br>
Credits: [blog one](https://towardsdatascience.com/deep-learning-for-object-detection-a-comprehensive-review-73930816d8d9)<br><br>
**Q4**: What is RPN and how does it works?<br>
**A4**: The best way to understand RPN is to see how it is mixed with R-CNN architecture. It can be described in 3 steps:<br>
**1.** At the beggining of Faster R-CNN we have pre-trained CNN to extract feature maps ([ImageNet](http://www.image-net.org/) for example). So input image goes through that CNN which outputs convolutional feature maps as can be seen below.<br>

<img src="https://qph2.c7.quoracdn.net/main-qimg-3fa20d817a121d31c641978114ff6f5d.webp"><br>
Credits: [blog three](https://qph2.c7.quoracdn.net/main-qimg-3fa20d817a121d31c641978114ff6f5d.webp)<br><br>

**2.** Having those feature maps, **sliding window** approach is used. We choose fixed size for a widow, for example 3x3(nxn) and run it through feature maps (with previously determied stride). For each sliding window, 9(k) anchors are generated like in the image below. They have different **aspect ratio** and **scale** but the same center coordinates.<br>

<img src="https://qph2.c7.quoracdn.net/main-qimg-5f23c56676fcf1369a836f625500c19e.webp"><br>
Credits: [blog three](https://qph2.c7.quoracdn.net/main-qimg-3fa20d817a121d31c641978114ff6f5d.webp)<br><br>

For each of those 9(k) anchors a value $p^*$ is computed which is **Intersection over Union** calculated like this:<br>

$IoU = \frac{Anchor \bigcap GroundTruthBox}{Anchor \bigcap GroudTruthBox}$

$p^{*}=\left\{\begin{array}{@{}ll@{}ll@{}}1, & \text{if}\ IoU > 0.7 \\ -1, & \text{if}\ IoU < 0.3 \\ 0, & \text{otherwise}\end{array}\right.$

**3.** Next those spatial feature maps, in that case 3x3(nxn), are fed to small network(RPN) which has to tasks:
1. classification: determine wheter spesific box contains an object (1 when there is an object, 0 when it is not)
2. regression: predict bounding boxes<br><br>
The regression layer has $4*9=36$ outputs, that is $(w, h, x, y)$ coordinates for each anchor.<br>
The classification layer has $2*9=18$ outputs, that is (probability of having object, probability of being a backgorund). <br><br>

After generating those proposals **Non-maximum suppresion(NMS)** is used to discard overlaping porposlals of the same object.<br>
**Q5**: How does NMS works?<br>
**A5**: First we leave only those proposals of the same object that has higer $IoU$ than some threshold (for example 0.5). Then select the proposal with the highest $IoU$ and discard others. In that way we left only "most probable" proposal.<b><br>

    
In the end we have a buch of object proposals with no class assigned to them, only proposals that most likely contain some object. Now we have classify those proposal areas into desired categories(classes). It is done in the same way as in Fast R-CNN, that is using **RoI polling layers** which are described in Fast R-CNN section above.<br>
    
Loss function of the whole model is a weighet sum of 4 losses! Two from RPN and antoher two from R-CNN.<br>
1. **RPN**<br>
    1.1 classification($CL_{RPN}$): two classes - (object, not-object)<br>
    1.2 regression($REG_{RPN})$: adjusting bounding box coordiates $(w, h, x, y)$<br>
2. **R-CNN**<br>
    2.1 classification($CL_{R-CNN}$): multiple classes - (cat, dog, bike ...., **background**)<br>
    2.2 regression($REG_{R-CNN}$: adjusting bouding box better than in RPN proposals

# TO DO

- Expand Faster R-CNN explanation above using this [link](https://www.quora.com/How-does-the-region-proposal-network-RPN-in-Faster-R-CNN-work) (easy) **DONE**
- Go through [that](http://www.telesens.co/2018/03/11/object-detection-and-classification-using-r-cnns/) blog post with very detailed tutorial how to implement Faster R-CNN in PyTorch (rather hard and time-consuming but very rewarding probably)
- Check out Mask R-CNN implementation [here](https://engineering.matterport.com/splash-of-color-instance-segmentation-with-mask-r-cnn-and-tensorflow-7c761e238b46)
- Useful site for labeling [data](http://www.robots.ox.ac.uk/~vgg/software/via/)
- One of the best explanation of Faster R-CNN along with training description [link](https://tryolabs.com/blog/2018/01/18/faster-r-cnn-down-the-rabbit-hole-of-modern-object-detection/)